# Converter datas pré gregorianas.


Necessita de 

    pip install convertdate

Exº Dehergne, entrada 628 Francisco Pérez 


 [http://timelink.uc.pt/mhk/china/id/rp-87](http://timelink.uc.pt/mhk/china/id/rp-87)

 M. 12 févr. 1583 (ancien style) = 22 février (nouveau style de la réforme introduite à Rome en oct. 1582),

In [2]:
from convertdate import julian
from convertdate import gregorian

julian.to_gregorian(1583,2,12)

(1583, 2, 22)

In [11]:
print(julian.to_gregorian(1582,9,24))
print(julian.to_gregorian(1582,10,5))

(1582, 10, 4)
(1582, 10, 15)


In [14]:
from convertdate import gregorian

gregorian.monthcalendar(1582,10)

[[None, None, None, None, None, 1, 2],
 [3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16],
 [17, 18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29, 30],
 [31, None, None, None, None, None]]

# Testing mhk_users database

In [2]:
from timelink.mhk.utilities import get_connection_string, get_engine
from sqlalchemy import MetaData, Table
from sqlalchemy.orm import declarative_base, sessionmaker

db = 'mhk_users'
mhk_users = MetaData()
Base = declarative_base()
connection_string = get_connection_string(db)
engine = get_engine(db)
engine.echo = False     # set to False to prevent SQL logging
Session = sessionmaker(bind=engine)
comments_table = Table('comments', mhk_users, autoload_with=engine)
users_table = Table('users', mhk_users, autoload_with=engine)
print('Connected to',db)


Connected to mhk_users


In [20]:
from sqlalchemy import select

print(comments_table.c)
stmt = select(comments_table.c.title,comments_table.c.text).where(comments_table.c.title.like('CORRIGIR%'))
with Session() as session:
    result = session.execute(stmt)
    for r in result:
        print(r.title)
        print('===================')
        for l in r.text.split('\n'):
            print('=',l)
        print()
       

ImmutableColumnCollection(comments.id, comments.db_name, comments.entity_id, comments.user_name, comments.user_ip, comments.email, comments.status, comments.title, comments.text, comments.created, comments.parent)
CORRIGIR data final
=             Id: 169434             C&oacute;digo de refer&ecirc;ncia:
= PT/AUC/ELU/UC-AUC/B/001-001/M/004236              Nome        :
= Lu&iacute;s Martins             Data inicial: 1538             Data
= final  : 1552             Filia&ccedil;&atilde;o: Br&aacute;s Martins
= Naturalidade: Cabe&ccedil;a de Vide             Faculdade:C&acirc;nones
= Matr&iacute;cula(s): 1538             03.01.1540              Provas:
= 03.08.1539             Provou Ouvir uma li&ccedil;&atilde;o em
= C&acirc;nones 1551-1552

CORRIGIR data final
=             Id: 240217             C&oacute;digo de refer&ecirc;ncia:
= PT/AUC/ELU/UC-AUC/B/001-001/F/000021              Nome        :
= Ant&oacute;nio Fagundes (padre)             Data inicial: 1540-09-30
= Data final  : 155

## Source files

In [ ]:
from sqlalchemy import select
from pathlib import Path

from timelink.mhk.models.base import Source


kleio_files = [f.stem for f in list(Path('../sources').rglob('*.cli'))]
print("Number of kleio_files:", len(kleio_files))

stmt = select(Source.id,Source.updated)

with Session() as session:
    imported_sources = session.execute(stmt)
    sources_in_db = [s.id for s in imported_sources]
    print("Number of imported files:",len(sources_in_db))
    print("Files not in the database:", len(set(kleio_files)-set(sources_in_db)))
    for source in sorted(list(set(kleio_files)-set(sources_in_db))):
        print(source,end=' ')
    print()
    print("Imported sources with no file found:", len(set(sources_in_db)-set(kleio_files)))
    for source in sorted(list(set(sources_in_db)-set(kleio_files))):
        print(source,end=' ')
    print()

## Linking to wikidata

Requires

        pip install sparqlwrapper

https://rdflib.github.io/sparqlwrapper/

### Students of the University of Coimbra born before the 20th century

In [2]:

query = """SELECT DISTINCT ?item ?itemLabel  ?instance_of ?instance_ofLabel   WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item p:P69 ?statement0.
      ?statement0 (ps:P69/(wdt:P279*)) wd:Q368643.
      ?item p:P569 ?statement_1.
      ?statement_1 psv:P569 ?statementValue_1.
      ?statementValue_1 wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?statementValue_1 wikibase:timeValue ?P569_1.
      FILTER(?P569_1 < "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    LIMIT 10
  }
}"""

In [44]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"



def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16917757'}, 'itemLabel': {'type': 'literal', 'value': 'Q16917757'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17272838'}, 'itemLabel': {'type': 'literal', 'value': 'Q17272838'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17280031'}, 'itemLabel': {'type': 'literal', 'value': 'Q17280031'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21290090'}, 'itemLabel': {'type': 'literal', 'value': 'Q21290090'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21290098'}, 'itemLabel': {'type': 'literal', 'value': 'Q21290098'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21290114'}, 'itemLabel': {'type': 'literal', 'value': 'Q21290114'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q21290141'}, 'itemLabel': {'type': 'literal', 'value': 'Q21290141'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.


        pip install pywikibot

https://www.wikidata.org/wiki/Wikidata:Pywikibot_-_Python_3_Tutorial/Data_Harvest

In [34]:
import pywikibot

claims_of_interest = ['P27']
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()
item = pywikibot.ItemPage(repo, "Q188916")

item_dict = item.get() #Get the item dictionary
labels = item_dict['labels']
label_pt = labels['pt']
print(label_pt)
clm_dict = item_dict["claims"] # Get the claim dictionary
for claim_id,claims in clm_dict.items():
    print(claim_id,end=", ")
    if claim_id in claims_of_interest:
        for claim in claims:
            print(claim.toJSON())
            clm_trgt = claim.getTarget()
            country_id = clm_trgt.getID()
            country_url = clm_trgt.full_url()
            country_info = pywikibot.ItemPage(repo, country_id)
            country_dict = country_info.get()
            country_labels = country_dict['labels']
            label_pt = country_labels['pt']
            print(label_pt)



Alexandre Rodrigues Ferreira
P214, P213, P21, P106, P569, P570, P31, P27, {'mainsnak': {'snaktype': 'value', 'property': 'P27', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 155}, 'type': 'wikibase-entityid'}}, 'type': 'statement', 'id': 'Q188916$AC8ACFF2-573A-4E06-B572-D84DB5C0126D', 'rank': 'normal'}
Brasil
P735, P1006, P648, P428, P2349, P2163, P950, P227, P586, P69, P244, P20, P19, P3241, P3987, P269, P1412, P463, P1871, P268, P6264, P1005, P18, P7859, P8034, P1207, P646, P7293, P1695, P9464, 

In [35]:
!pip install mkwikidata

You should consider upgrading via the '/Users/jrc/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [45]:
import mkwikidata

query_result = mkwikidata.run_query(query, params={ })
query_result

{'head': {'vars': ['item', 'itemLabel', 'instance_of', 'instance_ofLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q16917757'},
    'itemLabel': {'type': 'literal', 'value': 'Q16917757'}},
   {'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q17272838'},
    'itemLabel': {'type': 'literal', 'value': 'Q17272838'}},
   {'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q17280031'},
    'itemLabel': {'type': 'literal', 'value': 'Q17280031'}},
   {'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q21290090'},
    'itemLabel': {'type': 'literal', 'value': 'Q21290090'}},
   {'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q21290098'},
    'itemLabel': {'type': 'literal', 'value': 'Q21290098'}},
   {'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q21290114'},
    'itemLabel': {'type': 'literal', 'value': 'Q21290114'}},
   {'item': 

# qwikidata



In [46]:
!pip install qwikidata


  Using cached mypy_extensions-0.4.3-py2.py3-none-any.whl (4.5 kB)
You should consider upgrading via the '/Users/jrc/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [33]:

query = """SELECT DISTINCT ?item ?itemLabel  ?instance_of ?instance_ofLabel   WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item p:P69 ?statement0.
      ?statement0 (ps:P69/(wdt:P279*)) wd:Q368643.
      ?item p:P569 ?statement_1.
      ?statement_1 psv:P569 ?statementValue_1.
      ?statementValue_1 wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?statementValue_1 wikibase:timeValue ?P569_1.
      FILTER(?P569_1 < "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    LIMIT 500
  }
}"""

In [36]:
from datetime import datetime

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)

def value_of_prop(prop,propValue,cache=None):
    """ 
    propValue is ['mainsnak']['datavalue'] of a claim prop
    Returns the value of a property in a claim
    in a usefull way
    """
    prop_type = propValue['type']
    if prop_type == 'string':
        return propValue['value']
    # {'value': {'time': '+1841-07-30T00:00:00Z',
    #  'timezone': 0, 'before': 0, 'after': 0, 'precision': 11, 
    #  'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}, 'type': 'time'}
    elif prop_type == 'time':
        svalue = propValue['value']['time']
        date_value = datetime.strptime(svalue,"+%Y-%m-%dT%H:%M:%SZ")
        return date_value

    # P106: occupation/wikibase-entityid = 
    #               {'value': 
    #                   {'entity-type': 'item', 'numeric-id': 593644, 'id': 'Q593644'}, 'type': 'wikibase-entityid'}
    elif prop_type == 'wikibase-entityid':
        id = propValue['value']['id']
        if cache is not None and id in cache.keys():
            return cache[id]

        item_type = propValue['value']['entity-type']
        entity_info = get_entity_dict_from_api(id)
        names = [label['value'] for label in entity_info['labels'].values() if label['language'] in ['pt','en']]
        name = "; ".join(names)
        cache[id] = name
        return name

    else:
        return propValue

# https://www.wikidata.org/wiki/Property:P1559 in native language
props_interest = ['P1559','P1447','P27','P569','P570','P106','P19','P20','P1005','P102','P39','P166','P213','P214']
prop_labels = {}
cache = {}
estudantes = return_sparql_query_results(query)
for estudante in estudantes['results']['bindings']:
    estudanteLabel = estudante['itemLabel']['value']
    print(estudanteLabel)
    estudante_dict = get_entity_dict_from_api(estudanteLabel)
    names = [label['value'] for label in estudante_dict['labels'].values() if label['language'] in ['pt','en']]
    name = "; ".join(names)
    print()
    print(name)
    sites = []
    for site, site_info in estudante_dict['sitelinks'].items():
        if site in ['ptwiki','commonswiki']:
            print(f"  {site} {site_info['title']} {site_info['url']}")
        else:
            sites = sites + [site]
    print("   More sites:",sites)
    descs = [(desc['language'],desc['value']) for desc in estudante_dict['descriptions'].values()]
    print("   Alternate names in:",[n for n,v in descs])

    properties = estudante_dict['claims'].keys()
    not_interesting = []
    for prop in properties:
        if prop in prop_labels.keys():
            propLabel = prop_labels[prop]
        else:
            prop_dict = get_entity_dict_from_api(prop)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[prop] = propLabel
        if prop in props_interest:
            try:
                propValue = estudante_dict['claims'][prop][0]['mainsnak']['datavalue']
                print(f"  {prop}: {propLabel}/{propValue['type']} = {value_of_prop(prop,propValue,cache)}")
            except Exception as e:
                print("ERROR could not display value for ",prop)
        else:
            not_interesting = not_interesting + [f"{prop}:{propLabel}"]
    print("  Other properties not used:", not_interesting)


Q188916

Alexandre Rodrigues Ferreira; Alexandre Rodrigues Ferreira
  commonswiki Category:Alexandre Rodrigues Ferreira https://commons.wikimedia.org/wiki/Category:Alexandre_Rodrigues_Ferreira
  ptwiki Alexandre Rodrigues Ferreira https://pt.wikipedia.org/wiki/Alexandre_Rodrigues_Ferreira
   More sites: ['arwiki', 'arzwiki', 'enwiki', 'eowiki', 'frwiki', 'specieswiki']
   Alternate names in: ['fa', 'en', 'nl', 'de', 'pt-br', 'ml', 'pt', 'pap']
  P214: VIAF ID/string = 22230443
  P213: ISNI/string = 0000 0001 2277 3758
  P106: occupation/wikibase-entityid = explorer; explorador
  P569: date of birth/time = 1756-04-27 00:00:00
  P570: date of death/time = 1815-04-23 00:00:00
  P27: country of citizenship/wikibase-entityid = Brasil; Brazil
  P20: place of death/wikibase-entityid = Lisbon; Lisboa
  P19: place of birth/wikibase-entityid = Salvador; Salvador
  P1005: Portuguese National Library ID/string = 58614
  Other properties not used: ['P21:sex or gender', 'P31:instance of', 'P735:give

In [17]:
for p,desc in prop_labels.items():
    print(f" {p:8s} {desc}")

 P106     occupation
 P214     VIAF ID
 P213     ISNI
 P244     Library of Congress authority ID
 P227     GND ID
 P19      place of birth
 P20      place of death
 P27      country of citizenship
 P31      instance of
 P569     date of birth
 P570     date of death
 P18      image
 P646     Freebase ID
 P69      educated at
 P108     employer
 P735     given name
 P26      spouse
 P648     Open Library ID
 P3430    SNAC ARK ID
 P2381    Academic Tree ID
 P1412    languages spoken, written or signed
 P373     Commons category
 P1015    NORAF ID
 P1207    NUKAT ID
 P3360    Nobel Prize People Nomination ID
 P3987    SHARE Catalogue author ID
 P1005    Portuguese National Library ID
 P1006    Nationale Thesaurus voor Auteurs ID
 P269     IdRef ID
 P268     Bibliothèque nationale de France ID
 P21      sex or gender
 P185     doctoral student
 P691     NKCR AUT ID
 P1559    name in native language
 P271     CiNii author ID (books)
 P1343    described by source
 P7859    WorldCat Identitie